## Batch Inference
we will do batch inference using a model that has been trained. there are a couple of ways we can do this.

1. Continue in the training notebook right after training (easy but not practical)
2. The way we see below, more practical and transalatable to production

In [11]:
import boto3
import botocore
import sagemaker
import sys
import pandas as pd


bucket = sagemaker.Session().default_bucket()  # Feel free to change to another bucket you have access to
execution_role = sagemaker.get_execution_role()
region = boto3.Session().region_name


In [3]:
model_name='randomcutforest-2022-03-09-06-12-21-577'

In [6]:
prefix="input"
fileName="values_and_others.csv"

https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html#batch-transform-data-processing-example-select-csv

In [8]:
batch_input ='s3://{}/{}/{}'.format(bucket, prefix, fileName) # The location of thetest dataset
batch_output = 's3://{}/{}/batch-inference'.format(bucket, prefix) # The location to store the results of the batch transform job
transformer = sagemaker.transformer.Transformer(model_name=model_name,instance_count=1, instance_type='ml.m4.xlarge',output_path=batch_output,assemble_with="Line",accept="text/csv")
transformer.transform(data=batch_input,join_source="Input",input_filter="$[2:2]", data_type='S3Prefix', content_type='text/csv',split_type='Line')
transformer.wait()

....................................Docker entrypoint called with argument(s): serve
Running default environment configuration script
[03/09/2022 06:38:55 INFO 140392454883136] loaded entry point class algorithm.serve.server_config:config_api
[03/09/2022 06:38:55 INFO 140392454883136] loading entry points
[03/09/2022 06:38:55 INFO 140392454883136] Loaded iterator creator application/x-recordio-protobuf for content type ('application/x-recordio-protobuf', '1.0')
[03/09/2022 06:38:55 INFO 140392454883136] loaded request iterator application/json
[03/09/2022 06:38:55 INFO 140392454883136] loaded request iterator application/jsonlines
[03/09/2022 06:38:55 INFO 140392454883136] loaded request iterator application/x-recordio-protobuf
[03/09/2022 06:38:55 INFO 140392454883136] loaded request iterator text/csv
[03/09/2022 06:38:55 INFO 140392454883136] loaded response encoder application/json
[03/09/2022 06:38:55 INFO 140392454883136] loaded response encoder application/jsonlines
[03/09/2022 0

## post processing

What we are doing below is to read the output from batch inference to a dataframe, it will not have headers as we can't send it headers for batch inference, we will download, read into a dataframe and provide headers

*Note: in Productions you will have a post processing step to add in headers for readability if saving as a csv in S3*

In [9]:
!aws s3 cp s3://$bucket/$prefix/batch-inference/values_and_others.csv.out .

download: s3://sagemaker-us-east-1-240384047251/input/batch-inference/values_and_others.csv.out to ./values_and_others.csv.out


In [12]:
df_infer=pd.read_csv('values_and_others.csv.out', names=['timestamp','values','hour','score'])


In [13]:
df_infer.head()

,timestamp,values,hour,score
0,2017-01-01 00:00:00,14,0,0.959243
1,2017-01-01 00:15:00,39,0,0.959243
2,2017-01-01 00:30:00,75,0,0.959243
3,2017-01-01 00:45:00,116,0,0.959243
4,2017-01-01 01:00:00,124,1,0.808368
